<h1>Modeling of a Catheter <br>for Cardiac Ablation</h1>
<p>Daniel Newman <br>
<a href="mailto:danielnewman09@gmail.com">danielnewman09@gmail.com</a> <br>
<a href="http://danielnewman09.github.io">http://danielnewman09.github.io</a> <br>